In [1]:
from PIL import Image
import sys
import easyocr
import os
import re
import editdistance
import cv2
from matplotlib import pyplot as plt
from torchmetrics import WordErrorRate, CharErrorRate, MatchErrorRate

In [2]:
%matplotlib inline

In [3]:
dic_names = {}
for file in os.listdir("data/natation/"):
    if file[-4:] == ".txt":
        with open(os.path.join("data/natation/", file), 'r') as f:
            names = f.readlines()
            for n in names:
                complete_name = n.strip().split()[:-1]
                last_name = complete_name[:-1]
                first_name = complete_name[-1]
                if len(last_name) == 1:
                    last_name = ' '.join(complete_name[:-1])
                    if last_name in dic_names:
                        continue
                    else:
                        dic_names[last_name] = first_name
                else:
                    for ln in last_name:
                        if ln in dic_names: # ISSUE IF NAME WITH DIFFERENT FIRDT NAMES
                            continue
                        else:
                            dic_names[ln] = first_name

In [4]:
reader = easyocr.Reader(['en'])

CUDA not available - defaulting to CPU. Note: This module is much faster with a GPU.


In [5]:
def reformat_predictions(raw_predictions):
    return [pred[1] for pred in raw_predictions]

In [6]:
def get_prediction_last_name(prediction, dic_names, min_edit_distance=2):
    pattern = r'[0-9]'
    cprediction = re.sub(pattern, '', prediction.strip()).strip()
    last_names = []
    for comp in cprediction.split(' '):
        for n in dic_names.keys():
            if editdistance.eval(n, comp) <= min_edit_distance:
                last_names.append(comp)
    return last_names

In [7]:
def get_predictions_of_interest(predictions, dic_names):
    preds = []
    for pred in predictions:
        cpred = get_prediction_last_name(pred, dic_names)
        if len(cpred) == 0:
            continue
        elif len(cpred) == 1:
            preds.extend(cpred)
        else:
            cpred = ' '.join(cpred)
            preds.append(cpred)
    return preds

In [8]:
def get_ground_truth(path):
    ground_truth = []
    pattern = r'[0-9]'
    with open(path, "r") as f:
        ground_truth.extend([re.sub(pattern, '', y.strip()).strip(':., ') for y in f.readlines()])
    return ground_truth

In [9]:
def get_last_names_ground_truth(ground_truth):
    return [' '.join(gt.split(' ')[:-1]) for gt in ground_truth]

In [10]:
def compute_metrics(predictions_of_interest, last_names_ground_truth):
    mers = {yhat: 0 for yhat in predictions_of_interest}
    wers = {yhat: 0 for yhat in predictions_of_interest}
    cers = {yhat: 0 for yhat in predictions_of_interest}

    mer = MatchErrorRate()
    wer = WordErrorRate()
    cer = CharErrorRate()

    for yhat in predictions_of_interest:
        min_mer_error_rate = 1000
        min_wer_error_rate = 1000
        min_cer_error_rate = 1000
        for y in last_names_ground_truth:
            if mer(yhat, y) < min_mer_error_rate:
                min_mer_error_rate = mer(yhat, y)
            if wer(yhat, y) < min_wer_error_rate:
                min_wer_error_rate = wer(yhat, y)
            if cer(yhat, y) < min_cer_error_rate:
                min_cer_error_rate = cer(yhat, y)
        mers[yhat] = min_mer_error_rate.item()
        wers[yhat] = min_wer_error_rate.item()
        cers[yhat] = min_cer_error_rate.item()

    return mers, wers, cers

In [11]:
def compute_metrics_for_image(gt_path, im_path, dic_names):
    ground_truth = get_ground_truth(gt_path)
    last_names_ground_truth = get_last_names_ground_truth(ground_truth)

    predictions = reformat_predictions(reader.readtext(im_path))
    predictions = get_predictions_of_interest(predictions, dic_names)

    return compute_metrics(predictions, last_names_ground_truth)

In [12]:
def get_final_metric(gt_path, im_path, dic_names):
    ground_truth = get_ground_truth(gt_path)
    ground_truth = get_last_names_ground_truth(ground_truth)

    mers, wers, cers = compute_metrics_for_image(gt_path, im_path, dic_names)
    global_mer = sum(mers.values())/len(ground_truth)
    global_wer = sum(wers.values())/len(ground_truth)
    global_cer = sum(cers.values())/len(ground_truth)
    
    return (global_mer + global_wer + global_cer) / 3

In [13]:
data_path = 'data/natation/'
path_tuples = [(f"00000{i}.png", f"00000{i}.txt") for i in range(10)]
path_tuples.extend([(f"0000{i}.png", f"0000{i}.txt") for i in range(10, 15)])

In [14]:
def evaluate(data_path, path_tuples, dic_names):
    metric = []
    for i in range(len(path_tuples)):
        im_path = data_path + path_tuples[i][0]
        gt_path = data_path + path_tuples[i][1]
        metric.append(get_final_metric(gt_path, im_path, dic_names))
    return sum(metric)/len(metric)

In [15]:
evaluate(data_path, path_tuples, dic_names)

1.006104005200417